In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
from datetime import timedelta
import time

https://pandas.pydata.org/docs/reference/api/pandas.pivot_table.html

In [ ]:
df = pd.read_csv('/content/marathon_results_2016.csv')
df.head()

,Bib,Name,Age,M/F,City,State,Country,Citizen,Unnamed: 8,5K,10K,15K,20K,Half,25K,30K,35K,40K,Pace,Proj Time,Official Time,Overall,Gender,Division
0,5,"Hayle, Lemi Berhanu",21,M,Addis Ababa,NaN,ETH,NaN,NaN,0:15:47,0:31:20,0:47:07,1:03:14,1:06:45,1:19:15,1:34:17,1:50:24,2:05:59,0:05:04,2:12:45,2:12:45,1,1,1
1,1,"Desisa, Lelisa",26,M,Ambo,NaN,ETH,NaN,NaN,0:15:47,0:31:21,0:47:08,1:03:14,1:06:46,1:19:15,1:34:17,1:50:24,2:05:59,0:05:06,2:13:32,2:13:32,2,2,2
2,6,"Tsegay, Yemane Adhane",31,M,Addis Ababa,NaN,ETH,NaN,NaN,0:15:46,0:31:20,0:47:07,1:03:13,1:06:44,1:19:15,1:34:45,1:50:48,2:06:47,0:05:07,2:14:02,2:14:02,3,3,3
3,11,"Korir, Wesley",33,M,Kitale,NaN,KEN,NaN,NaN,0:15:46,0:31:21,0:47:07,1:03:14,1:06:46,1:19:16,1:34:45,1:50:48,2:06:47,0:05:07,2:14:05,2:14:05,4,4,4
4,14,"Lonyangata, Paul",23,M,Eldoret,NaN,KEN,NaN,NaN,0:15:46,0:31:21,0:47:08,1:03:14,1:06:46,1:19:18,1:34:46,1:51:30,2:08:11,0:05:11,2:15:45,2:15:45,5,5,5


In [ ]:
df = pd.read_csv('/content/marathon_results_2016.csv', index_col='Bib',
usecols=['Bib','40K','Half','Pace','Age','M/F','Country','State','City'])
df.head()

,Age,M/F,City,State,Country,Half,40K,Pace
Bib,,,,,,,,
5,21,M,Addis Ababa,NaN,ETH,1:06:45,2:05:59,0:05:04
1,26,M,Ambo,NaN,ETH,1:06:46,2:05:59,0:05:06
6,31,M,Addis Ababa,NaN,ETH,1:06:44,2:06:47,0:05:07
11,33,M,Kitale,NaN,KEN,1:06:46,2:06:47,0:05:07
14,23,M,Eldoret,NaN,KEN,1:06:46,2:08:11,0:05:11


Konwersja kolumn do typu timedelta

In [ ]:
df['40K'] = df['40K'].apply(pd.to_timedelta)
df['Half'] = df['Half'].apply(pd.to_timedelta)
df['TotalSeconds'] = df['40K'].apply(lambda x: timedelta.total_seconds(x))
df['HalfSeconds'] = df['Half'].apply(lambda x: timedelta.total_seconds(x))
df.head(1000)

,Age,M/F,City,State,Country,Half,40K,Pace,TotalSeconds,HalfSeconds
Bib,,,,,,,,,,
5,21,M,Addis Ababa,NaN,ETH,0 days 01:06:45,0 days 02:05:59,0:05:04,7559.0,4005.0
1,26,M,Ambo,NaN,ETH,0 days 01:06:46,0 days 02:05:59,0:05:06,7559.0,4006.0
6,31,M,Addis Ababa,NaN,ETH,0 days 01:06:44,0 days 02:06:47,0:05:07,7607.0,4004.0
11,33,M,Kitale,NaN,KEN,0 days 01:06:46,0 days 02:06:47,0:05:07,7607.0,4006.0
14,23,M,Eldoret,NaN,KEN,0 days 01:06:46,0 days 02:08:11,0:05:11,7691.0,4006.0
...,...,...,...,...,...,...,...,...,...,...
766,40,M,Philadelphia,PA,USA,0 days 01:17:56,0 days 02:43:28,0:06:44,9808.0,4676.0
2004,41,M,Brick,NJ,USA,0 days 01:27:24,0 days 02:47:19,0:06:44,10039.0,5244.0
1865,40,M,Andover,MA,USA,0 days 01:25:57,0 days 02:47:21,0:06:44,10041.0,5157.0


Wyświetl tabelę przestawną, która pozwoli porównać średni czas biegu na dystansie 40 km dla
zawodników w różnym wieku z podziałem na mężczyzn i kobiety: w wierszach ma być wiek zawodników
(kolumna Age), w kolumnach informacja o płci (kolumna M/F), w komórkach średnia z czasu
TotalSeconds

In [ ]:
df.pivot_table(index="Age",columns="M/F",values="TotalSeconds", aggfunc='mean').head()

M/F,F,M
Age,,
18,16050.666667,14554.615385
19,15351.958333,12156.529412
20,13835.931818,12111.100000
21,14681.027027,12408.360465
22,14366.421053,11872.666667


Do poprzedniej tabeli dodaj również dane o średnim czasie HalfSeconds

In [ ]:
df.pivot_table(index="Age",columns="M/F",values=["HalfSeconds","TotalSeconds"]).head()

HalfSeconds               TotalSeconds              
M/F            F            M             F             M
Age                                                      
18   7999.888889  7078.538462  16050.666667  14554.615385
19   7473.666667  5817.764706  15351.958333  12156.529412
20   6871.045455  5716.800000  13835.931818  12111.100000
21   7148.554054  5929.627907  14681.027027  12408.360465
22   7091.097744  5751.000000  14366.421053  11872.666667

Zmień kolejność poziomów dla kolumn. Najpierw ma być dokonany podział na mężczyzn i kobiety, a
dopiero potem rozróżnienie średnich czasów na dystansie 20 i 40 km

In [ ]:
df.pivot_table(index="Age",columns="M/F",values=["HalfSeconds","TotalSeconds"]).swaplevel(axis=1).head()

M/F,F,M,F,M
,HalfSeconds,HalfSeconds,TotalSeconds,TotalSeconds
Age,,,,
18,7999.888889,7078.538462,16050.666667,14554.615385
19,7473.666667,5817.764706,15351.958333,12156.529412
20,6871.045455,5716.800000,13835.931818,12111.100000
21,7148.554054,5929.627907,14681.027027,12408.360465
22,7091.097744,5751.000000,14366.421053,11872.666667


Zmień sortowanie kolumn w tabeli z poprzedniego przykładu tak, aby nagłówek dla kobiet i mężczyzn
pojawił się tylko jeden raz.

In [ ]:
df.pivot_table(index="Age",columns="M/F",values=["HalfSeconds","TotalSeconds"]).swaplevel(axis=1).sort_index(axis=1).head()

M/F            F                          M              
     HalfSeconds  TotalSeconds  HalfSeconds  TotalSeconds
Age                                                      
18   7999.888889  16050.666667  7078.538462  14554.615385
19   7473.666667  15351.958333  5817.764706  12156.529412
20   6871.045455  13835.931818  5716.800000  12111.100000
21   7148.554054  14681.027027  5929.627907  12408.360465
22   7091.097744  14366.421053  5751.000000  11872.666667

A właściwie to ilu było zawodników? Zbuduj tabelę przestawną, która zaprezentuje w wierszach wiek
uczestników, a w kolumnie ilość uczestników z podziałem na płeć. Do liczenia zawodników może być
wykorzystane dowolne z dostępnych pól.

In [ ]:
df.pivot_table(index="Age",columns="M/F",aggfunc='count',values='City').head()

M/F,F,M
Age,,
18,9.0,13.0
19,24.0,17.0
20,44.0,40.0
21,74.0,86.0
22,133.0,93.0


Zmień poprzednie polecenie tak, aby wartości były wyznaczane bez podziału na kobiety i mężczyzn

In [ ]:
df.pivot_table(index="Age",aggfunc='count',values='City').head()

,City
Age,
18,22
19,41
20,84
21,160
22,226


Zbuduj tabelę, która policzy ilość uczestników z różnych krajów. Kraje mają się znaleźć w wierszach.

Zmien poprzednie polecenie tak, aby wiersze były posortowane wg ilości zawodników z poszczególnych
krajów malejąco.

In [ ]:
df.pivot_table(index="Country",aggfunc='count',values='City').sort_values('City', ascending=False).head()

,City
Country,
USA,21649
CAN,2134
GBR,366
MEX,252
JPN,185


Zbuduj tabelę przestawną, która będzie prezentować średni czas biegu uzyskany przez reprezentatów
poszczególnych miast. Tabela ma posiadać multiindeks w oparciu o kolumny County, State, City

In [ ]:
df.pivot_table(index=["Country","State","City"],aggfunc='mean',values='TotalSeconds').head()

TotalSeconds
Country State City                   
CAN     AB    Airdrie    11211.000000
              Ardrossan  12644.000000
              Calgary    12546.186813
              Cochrane   12879.500000
              Devon      14414.500000

Posortuj dane z poprzedniego polecenia wg czasu malejąco. W ten sposób wyznaczysz miasto z
najgorszymi średnimi wynikami.

In [ ]:
df.pivot_table(index=["Country","State","City"],aggfunc='mean',values='TotalSeconds').sort_values('TotalSeconds', ascending=False).head()

TotalSeconds
Country State City                      
USA     WI    Reedsburg          28461.0
        TX    Justin             23816.0
        MS    Olive Branch       21878.0
        FL    Estero             21468.0
        TX    Fresno             20919.0